In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, Image
import matplotlib
from matplotlib import font_manager, rc

import platform
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline
import warnings

In [2]:

import pandas as pd
tmp = []
tmp = pd.DataFrame()
gas = pd.read_csv( '한국가스공사_시간별 공급량_20181231.csv', encoding='cp949' )
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

for i in range(13,19,1):
  i=str(i)+".csv"
  for j in range(7):
    tmp = pd.concat( [tmp , pd.read_csv( i, encoding='cp949')])
 

In [3]:
tmp
# tmp1 = pd.read_csv('17.csv', encoding='cp949')
# tmp1['일시'] = pd.a = []
# total_2017A_p = total_2017A.loc[ total_2017A['연월일시'].values not in a, :]to_datetime(tmp1['일시'])
# tmp1.interpolate()

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),적설(cm),지면온도(°C),Unnamed: 0
0,108,서울,2013-01-01 00:00,-8.3,NaN,3.6,56.0,NaN,6.4,-3.3,NaN
1,108,서울,2013-01-01 01:00,-8.5,NaN,4.0,57.0,NaN,6.4,-3.4,NaN
2,108,서울,2013-01-01 02:00,-8.4,NaN,4.0,60.0,NaN,6.4,-3.4,NaN
3,108,서울,2013-01-01 03:00,-8.1,NaN,3.4,58.0,NaN,6.4,-3.4,NaN
4,108,서울,2013-01-01 04:00,-8.2,NaN,3.8,58.0,NaN,6.4,-3.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8755,108,서울,2018-12-31 19:00,-3.1,NaN,2.1,43.0,NaN,NaN,-2.2,NaN
8756,108,서울,2018-12-31 20:00,-3.7,NaN,0.8,39.0,NaN,NaN,-3.0,NaN
8757,108,서울,2018-12-31 21:00,-4.6,NaN,1.1,44.0,NaN,NaN,-4.1,NaN
8758,108,서울,2018-12-31 22:00,-5.4,NaN,1.3,46.0,NaN,NaN,-5.0,NaN


In [4]:
gas['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [5]:
gas.iloc[[(183960+(31+29)*24)+366*24]]

,연월일,시간,구분,공급량
194184,2016-03-01,1,B,1800.053


In [6]:

gas['연월일'] = pd.to_datetime(gas['연월일'])
tmp['일시'] = pd.to_datetime(tmp['일시'])
tmp.reset_index(inplace=True)
tmp.tail()


,index,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),적설(cm),지면온도(°C),Unnamed: 0
368083,8755,108,서울,2018-12-31 19:00:00,-3.1,NaN,2.1,43.0,NaN,NaN,-2.2,NaN
368084,8756,108,서울,2018-12-31 20:00:00,-3.7,NaN,0.8,39.0,NaN,NaN,-3.0,NaN
368085,8757,108,서울,2018-12-31 21:00:00,-4.6,NaN,1.1,44.0,NaN,NaN,-4.1,NaN
368086,8758,108,서울,2018-12-31 22:00:00,-5.4,NaN,1.3,46.0,NaN,NaN,-5.0,NaN
368087,8759,108,서울,2018-12-31 23:00:00,-5.2,NaN,1.6,47.0,NaN,NaN,-5.1,NaN


In [7]:

tmp.shape[0]
tmp.drop('index', axis = 1, inplace=True)
tmp.drop('일시', axis = 1, inplace=True)
tmp.drop('Unnamed: 0', axis = 1, inplace=True)

tmp.tail()


,지점,지점명,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),적설(cm),지면온도(°C)
368083,108,서울,-3.1,NaN,2.1,43.0,NaN,NaN,-2.2
368084,108,서울,-3.7,NaN,0.8,39.0,NaN,NaN,-3.0
368085,108,서울,-4.6,NaN,1.1,44.0,NaN,NaN,-4.1
368086,108,서울,-5.4,NaN,1.3,46.0,NaN,NaN,-5.0
368087,108,서울,-5.2,NaN,1.6,47.0,NaN,NaN,-5.1


In [8]:
gas=gas.join(tmp) # gas , tmp 합치기

In [9]:
gas.fillna(0,inplace=True)
gas.tail()

,연월일,시간,구분,공급량,지점,지점명,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),적설(cm),지면온도(°C)
368083,2018-12-31,20,H,681.033,108,서울,-3.1,0.0,2.1,43.0,0.0,0.0,-2.2
368084,2018-12-31,21,H,669.961,108,서울,-3.7,0.0,0.8,39.0,0.0,0.0,-3.0
368085,2018-12-31,22,H,657.941,108,서울,-4.6,0.0,1.1,44.0,0.0,0.0,-4.1
368086,2018-12-31,23,H,610.953,108,서울,-5.4,0.0,1.3,46.0,0.0,0.0,-5.0
368087,2018-12-31,24,H,560.896,108,서울,-5.2,0.0,1.6,47.0,0.0,0.0,-5.1


In [10]:
gas.drop('강수량(mm)',axis = 1,inplace=True)
gas.drop('풍속(m/s)',axis = 1,inplace=True)
gas.drop('습도(%)',axis = 1,inplace=True)
gas.drop('일조(hr)',inplace=True,axis = 1)
gas.drop('적설(cm)',inplace=True,axis = 1)
gas.drop('지면온도(°C)',inplace=True,axis = 1)


In [11]:
gas.head()

,연월일,시간,구분,공급량,지점,지점명,기온(°C)
0,2013-01-01,1,A,2497.129,108,서울,-8.3
1,2013-01-01,2,A,2363.265,108,서울,-8.5
2,2013-01-01,3,A,2258.505,108,서울,-8.4
3,2013-01-01,4,A,2243.969,108,서울,-8.1
4,2013-01-01,5,A,2344.105,108,서울,-8.2


In [12]:
gas.shape[0]

368088

In [13]:
gas['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [14]:
gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   연월일     368088 non-null  datetime64[ns]
 1   시간      368088 non-null  int64         
 2   구분      368088 non-null  object        
 3   공급량     368088 non-null  float64       
 4   지점      368088 non-null  int64         
 5   지점명     368088 non-null  object        
 6   기온(°C)  368088 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 19.7+ MB


In [15]:
gas.head() #공급량 예측한 것을 온도 예측하는 데에 사용, 또 온도 예측한 것을 공급량 예측에 사용 -> 성능 향상이 없을 때 까지 계속 반복

,연월일,시간,구분,공급량,지점,지점명,기온(°C)
0,2013-01-01,1,A,2497.129,108,서울,-8.3
1,2013-01-01,2,A,2363.265,108,서울,-8.5
2,2013-01-01,3,A,2258.505,108,서울,-8.4
3,2013-01-01,4,A,2243.969,108,서울,-8.1
4,2013-01-01,5,A,2344.105,108,서울,-8.2


In [16]:

gas['Year'] = gas['연월일'].dt.year
gas['Month'] = gas['연월일'].dt.month
gas['Day'] = gas['연월일'].dt.day
gas['weekday'] = gas['연월일'].dt.weekday

gas.head() , gas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   연월일      368088 non-null  datetime64[ns]
 1   시간       368088 non-null  int64         
 2   구분       368088 non-null  object        
 3   공급량      368088 non-null  float64       
 4   지점       368088 non-null  int64         
 5   지점명      368088 non-null  object        
 6   기온(°C)   368088 non-null  float64       
 7   Year     368088 non-null  int64         
 8   Month    368088 non-null  int64         
 9   Day      368088 non-null  int64         
 10  weekday  368088 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6), object(2)
memory usage: 30.9+ MB


(         연월일  시간 구분       공급량   지점 지점명  기온(°C)  Year  Month  Day  weekday
 0 2013-01-01   1  A  2497.129  108  서울    -8.3  2013      1    1        1
 1 2013-01-01   2  A  2363.265  108  서울    -8.5  2013      1    1        1
 2 2013-01-01   3  A  2258.505  108  서울    -8.4  2013      1    1        1
 3 2013-01-01   4  A  2243.969  108  서울    -8.1  2013      1    1        1
 4 2013-01-01   5  A  2344.105  108  서울    -8.2  2013      1    1        1,
 None)

In [17]:
# # GrLivArea / SalePirce간 scatter plot 확인
# var = 'GrLivArea'
# data = pd.concat([gas['SalePrice'], gas[var]], axis=1)
# data.plot.scatter(x=var, y='gas', ylim=(0,800000));

# data = pd.concat([gas['SalePrice'], train[var]], axis=1)
# data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [18]:
#-------------------------------------------------------------기온예측 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
gas['공급량'] = np.log1p(gas['공급량']) #정규화
gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   연월일      368088 non-null  datetime64[ns]
 1   시간       368088 non-null  int64         
 2   구분       368088 non-null  object        
 3   공급량      368088 non-null  float64       
 4   지점       368088 non-null  int64         
 5   지점명      368088 non-null  object        
 6   기온(°C)   368088 non-null  float64       
 7   Year     368088 non-null  int64         
 8   Month    368088 non-null  int64         
 9   Day      368088 non-null  int64         
 10  weekday  368088 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6), object(2)
memory usage: 30.9+ MB


In [19]:
d_map = {}
for i, d in enumerate(gas['구분'].unique()):
    d_map[d] = i
gas['구분'] = gas['구분'].map(d_map)

In [20]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]
train = gas[gas['Year'].isin(train_years)]
val = gas[gas['Year'].isin(val_years)]


In [21]:
features = ['구분', 'Month', 'Day', 'weekday', '시간']
train_x = train[features]
train_y = train['기온(°C)']

val_x = val[features]
val_y = val['기온(°C)']

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 3.51847
[40]	valid_0's l1: 3.13065
[60]	valid_0's l1: 3.10134
Early stopping, best iteration is:
[58]	valid_0's l1: 3.0972


In [23]:
gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   연월일      368088 non-null  datetime64[ns]
 1   시간       368088 non-null  int64         
 2   구분       368088 non-null  int64         
 3   공급량      368088 non-null  float64       
 4   지점       368088 non-null  int64         
 5   지점명      368088 non-null  object        
 6   기온(°C)   368088 non-null  float64       
 7   Year     368088 non-null  int64         
 8   Month    368088 non-null  int64         
 9   Day      368088 non-null  int64         
 10  weekday  368088 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(7), object(1)
memory usage: 30.9+ MB


In [24]:

d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 3.51847
[40]	valid_0's l1: 3.13065
[60]	valid_0's l1: 3.10134
Early stopping, best iteration is:
[58]	valid_0's l1: 3.0972


----------------------------------------

In [25]:
# tmp17 = pd.read_csv('17.csv', encoding='cp949')

In [26]:
# 빈데이터 찾기, 추가


# tmp17 = pd.read_csv('17.csv', encoding='cp949')
# new_data = [108,'서울',0,0,0,0,0,0,0,0] # 넣을 임시데이터

# j=0
# count = 0
# for i in range(len(tmp17['일시'])) :
#   st=str(tmp17['일시'][count])
#   count+=1
#   if(count > len(tmp17['일시'])):
#       break
#   time = format(j%24,'02') # 있어야 할 시간 카운트

#   if st.find(time+":00") == -1 :
#     j+=1
#     new_data[2] = st[:10] +" "+time+":00"
#     tmp17.loc[count-1/(sum+1)] = new_data
#     tmp17.sort_index(inplace=True)
   
#     if (int( time ) - int( st[11:13] )) < -1:
#       count = count - 1
#       continue
#   j+=1
# tmp17.reset_index(inplace=True)
# # 연속된 빈값 3개 때문에 해결이 미뤄지는중, 빈값에 추가하고 빈값부터 검사하는 작업이 필요

In [27]:
# # 빈데이터 찾기, 추가


# tmp17 = pd.read_csv('17.csv', encoding='cp949')
# new_data = [108,'서울',0,0,0,0,0,0,0,0] # 넣을 임시데이터

# j=0
# sum = 0
# count = 0
# for i in range(len(tmp17['일시'])) :
#   st=str(tmp17['일시'][count])
#   count+=1
#   if(count > len(tmp17['일시'])):
#       break
#   time = format(j%24,'02') # 있어야 할 시간 카운트

#   if st.find(time+":00") == -1 :
#     print(" =========================== ")
#     print("\n결측치 생긴위치 :                   ",st)
#     print("결측치 위치에 있어야할 시간 추가 :  ", st[:10] +" "+time+":00 \n")
#     j+=1
#     sum +=1
#     new_data[2] = st[:10] +" "+time+":00"
#     tmp17.loc[count-1/(sum+1)] = new_data
#     tmp17.sort_index()
#     # print(count,"/")
#     # print(count-1/(sum+1))
#     # print(tmp17.loc[(count-1/(sum+1))])
#     tmp17.sort_index(inplace=True)
#     print(tmp17.shape)
#     if (int( time ) - int( st[11:13] )) < -1:
#       print("  연속")
#       count = count - 1
#       continue
#   j+=1

# tmp17.reset_index(inplace=True)

# # 연속된 빈값 3개 때문에 해결이 미뤄지는중, 빈값에 추가하고 빈값부터 검사하는 작업이 필요
# print("\n------------------------------")
# print("결측치 총 개수 :",sum)
# tmp17.info()

In [28]:
# tmp17.drop('index', axis = 1, inplace=True)
# tmp17[tmp17['기온(°C)']==0]

In [29]:
# tmp17.to_csv("modi_17.csv",encoding='euc-kr') # 한글이 깨지는 이슈 -> 인코딩 매개변수로 해결

In [30]:
gas

,연월일,시간,구분,공급량,지점,지점명,기온(°C),Year,Month,Day,weekday
0,2013-01-01,1,0,7.823297,108,서울,-8.3,2013,1,1,1
1,2013-01-01,2,0,7.768222,108,서울,-8.5,2013,1,1,1
2,2013-01-01,3,0,7.722901,108,서울,-8.4,2013,1,1,1
3,2013-01-01,4,0,7.716447,108,서울,-8.1,2013,1,1,1
4,2013-01-01,5,0,7.760085,108,서울,-8.2,2013,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,6.525078,108,서울,-3.1,2018,12,31,0
368084,2018-12-31,21,6,6.508711,108,서울,-3.7,2018,12,31,0
368085,2018-12-31,22,6,6.490634,108,서울,-4.6,2018,12,31,0
368086,2018-12-31,23,6,6.416655,108,서울,-5.4,2018,12,31,0


In [31]:
gas

,연월일,시간,구분,공급량,지점,지점명,기온(°C),Year,Month,Day,weekday
0,2013-01-01,1,0,7.823297,108,서울,-8.3,2013,1,1,1
1,2013-01-01,2,0,7.768222,108,서울,-8.5,2013,1,1,1
2,2013-01-01,3,0,7.722901,108,서울,-8.4,2013,1,1,1
3,2013-01-01,4,0,7.716447,108,서울,-8.1,2013,1,1,1
4,2013-01-01,5,0,7.760085,108,서울,-8.2,2013,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,6.525078,108,서울,-3.1,2018,12,31,0
368084,2018-12-31,21,6,6.508711,108,서울,-3.7,2018,12,31,0
368085,2018-12-31,22,6,6.490634,108,서울,-4.6,2018,12,31,0
368086,2018-12-31,23,6,6.416655,108,서울,-5.4,2018,12,31,0


In [32]:
gas.drop(['지점','지점명'],axis=1,inplace=True)

In [33]:
cat_columns = [c for c, t in zip(gas.dtypes.index, gas.dtypes) if t == 'O'] 
num_columns = [c for c    in gas.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
[]

Numeric Columns: 
['연월일', '시간', '구분', '공급량', '기온(°C)', 'Year', 'Month', 'Day', 'weekday']


In [34]:
numeric_data = gas[num_columns]

In [35]:
test['일자|시간|구분'] = test['일자|시간|구분'].astype('str')
test_split = 	test['일자|시간|구분'].str.split(' ')

test['일자'] = test_split.str.get(0)
test['시간'] = test_split.str.get(1).astype(int)
test['구분'] = test_split.str.get(2)
test.drop('일자|시간|구분',axis=1,inplace=True)

test['일자'] = pd.to_datetime(test['일자'])
test['Year'] = test['일자'].dt.year
test['Month'] = test['일자'].dt.month
test['Day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday
test['구분'] = test['구분'].map(d_map)

In [36]:
test_x = test[features]
test_x

,구분,Month,Day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [37]:
preds = model.predict(test_x)
preds

array([-3.25407296, -3.60819802, -3.93580978, ...,  9.1032642 ,
        8.58442376,  8.36300935])

In [38]:
test["기온(°C)"] = preds

test.head()

,일자,시간,구분,Year,Month,Day,weekday,기온(°C)
0,2019-01-01,1,0,2019,1,1,1,-3.254073
1,2019-01-01,2,0,2019,1,1,1,-3.608198
2,2019-01-01,3,0,2019,1,1,1,-3.935810
3,2019-01-01,4,0,2019,1,1,1,-4.113031
4,2019-01-01,5,0,2019,1,1,1,-4.464113


In [39]:
test

,일자,시간,구분,Year,Month,Day,weekday,기온(°C)
0,2019-01-01,1,0,2019,1,1,1,-3.254073
1,2019-01-01,2,0,2019,1,1,1,-3.608198
2,2019-01-01,3,0,2019,1,1,1,-3.935810
3,2019-01-01,4,0,2019,1,1,1,-4.113031
4,2019-01-01,5,0,2019,1,1,1,-4.464113
...,...,...,...,...,...,...,...,...
15115,2019-03-31,20,6,2019,3,31,6,10.672685
15116,2019-03-31,21,6,2019,3,31,6,9.888458
15117,2019-03-31,22,6,2019,3,31,6,9.103264
15118,2019-03-31,23,6,2019,3,31,6,8.584424


In [40]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = gas[gas['Year'].isin(train_years)]
val = gas[gas['Year'].isin(val_years)]

features = ['구분', 'Month','Day', '시간', '기온(°C)']
X_train = train[features]
y_train = train['공급량']

X_test = val[features]
Y_test = val['공급량']

In [41]:
pd.DataFrame(np.log1p(gas['기온(°C)'])).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   기온(°C)  320201 non-null  float64
dtypes: float64(1)
memory usage: 2.8 MB


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [42]:
gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   연월일      368088 non-null  datetime64[ns]
 1   시간       368088 non-null  int64         
 2   구분       368088 non-null  int64         
 3   공급량      368088 non-null  float64       
 4   기온(°C)   368088 non-null  float64       
 5   Year     368088 non-null  int64         
 6   Month    368088 non-null  int64         
 7   Day      368088 non-null  int64         
 8   weekday  368088 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 25.3 MB


In [43]:
X_train = gas[features]
X_test = test[features]
y_train = gas['공급량']
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(368088, 5)
(15120, 5)
(368088,)


In [44]:
# ------------------------------------------------ 학습시키기

In [45]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV,LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score

In [46]:
#Validation function

n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [47]:

gb = GradientBoostingRegressor(n_estimators=250,random_state=0)

xgboost = xgb.XGBRegressor(n_estimators=250,random_state=0,nthread = -1)

lightgb = lgb.LGBMRegressor(n_estimators=250,random_state=0)

rf = RandomForestRegressor(n_estimators=250,random_state=0)

ridge = RidgeCV(cv = 10)
ridge = make_pipeline(RobustScaler(),ridge)


In [48]:
lr_final = LogisticRegression(C=10)

In [49]:
# score = rmsle_cv(gb)
# gb_score = score.mean()
# print("GradientBoostingRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

# score = rmsle_cv(xgboost)
# xgb_score = score.mean()
# print("XGBRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

# score = rmsle_cv(rf)
# print("rf score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

# score = rmsle_cv(ridge)
# print("ridge score: {:.4f} ({:.4f})".format(score.mean(), score.std())) 

# score = rmsle_cv(lightgb)
# lgb_score = score.mean()
# print("LGBMRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [50]:
from mlxtend.regressor import StackingCVRegressor

In [51]:
stack = StackingCVRegressor(regressors=(gb, xgboost, lightgb, rf, ridge),#svr
                                meta_regressor=xgboost,
                                use_features_in_secondary=True
                            )

In [52]:
# score = rmsle_cv(stack)
# print("stack score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [53]:

stack_model = stack.fit(np.array(X_train), y_train)


[00:35:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:36:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:36:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:36:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:56:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:58:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [54]:

pred = stack_model.predict(np.array(X_test))
# pd_test_pred = pd.DataFrame({'Id': Id_test, '공급량': np.expm1(pred)})
# pd_test_pred.to_csv('submission.csv', index=False)


In [55]:
np.expm1(pred)

array([1884.3696 , 1673.61   , 1611.3892 , ...,  351.09387,  324.2883 ,
        300.8302 ], dtype=float32)

In [56]:
sub['공급량'] = np.expm1(pred)
sub.to_csv('electric_baseline.csv', index=False)

In [57]:
# gb, xgboost, lightgb, rf, ridge

In [58]:

xgboost_model = xgboost.fit(X_train, y_train)
lightgb_model = lightgb.fit(X_train, y_train)
rf_model = rf.fit(X_train, y_train)
ridge_model = ridge.fit(X_train, y_train)


[01:03:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [59]:
from sklearn.svm import SVR

In [60]:
# 블랜딩은 오버피팅에 조금더 견고한 결과물을 만들어 낸다고 한다.
def blended_predictions(X):
    return ((0.13 * ridge_model.predict(X)) + \
            (0.2 * xgboost_model.predict(X)) + \
            (0.17 * lightgb_model.predict(X)) + \
            (0.5 * stack_model.predict(np.array(X))))

In [61]:
pred = blended_predictions(X_test)
sub['공급량'] = np.expm1(pred)
sub.to_csv('electric_baseline.csv', index=False)

In [70]:
sub['공급량']

0        1751.944100
1        1596.859855
2        1541.437858
3        1556.398265
4        1706.270308
            ...     
15115     384.304554
15116     394.137106
15117     390.937408
15118     366.258761
15119     343.798108
Name: 공급량, Length: 15120, dtype: float64

In [62]:
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_absolute_error

# # 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
# def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds=7 ):
#     # 지정된 n_folds값으로 KFold 생성.
#     kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
#     #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
#     train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
#     test_pred = np.zeros((X_test_n.shape[0],n_folds))
#     print(model.__class__.__name__ , ' model 시작 ')
    
#     for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
#         #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
#         print('\t 폴드 세트: ',folder_counter,' 시작 ')
#         X_tr = X_train_n.iloc[train_index] 
#         y_tr = y_train_n.iloc[train_index] 
#         X_te = X_train_n.iloc[valid_index]  
        
#         #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
#         model.fit(X_tr , y_tr)       
#         #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
#         train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
#         #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
#         test_pred[:, folder_counter] = model.predict(X_test_n)
            
#     # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
#     test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
#     #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
#     return train_fold_pred , test_pred_mean

In [63]:
# # CV스태킹 알고리즘 각 모델에 적용

# gb_train, gb_test = get_stacking_base_datasets(gb, X_train, y_train, X_test, 5)
# rf_train, rf_test = get_stacking_base_datasets(rf, X_train, y_train, X_test, 5)


In [64]:
# xgb_train, xgb_test = get_stacking_base_datasets(xgboost, X_train, y_train, X_test,  5)    
# ridge_train, ridge_test = get_stacking_base_datasets(ridge, X_train, y_train, X_test,  5)  



# # CV스태킹 알고리즘 결과로 메타 모델 학습/시험에 필요한 result_a result_b 만들기 
# Stack_final_X_train = np.concatenate((gb_train,rf_train,xgb_train, ridge_train), axis=1)
# Stack_final_X_test = np.concatenate((gb_test,rf_test,xgb_test,ridge_test), axis=1)



In [65]:
# Stack_final_X_train.shape

In [66]:
# # 메타 모델 학습
# LGBM.fit(Stack_final_X_train, y_train)

# stack_final = LGBM.predict(Stack_final_X_test)
# stack_final


In [67]:
# from mlxtend.regressor import StackingCVRegressor

In [68]:
# # 6개 모델 위에 xgboost 모델을 하나 더올려 stacking을 한다.
# stack = StackingCVRegressor(regressors=(gb, xgboost, lightgb, lasso, ridge),#svr
#                                 meta_regressor=xgboost,
#                                 use_features_in_secondary=True,
#                                 )

In [69]:
# score = rmsle_cv(stack)
print("stack score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

NameError: ignored

In [ ]:
# stack_model = stack.fit(np.array(X_train), y_train)

In [ ]:
# pred = stack_model.predict(np.array(X_test))
# # pd_test_pred = pd.DataFrame({'Id': Id_test, 'SalePrice': np.expm1(pred)})
# pd_test_pred.to_csv('submission.csv', index=False)

In [ ]:
# test_x = test[features]
# test_x

In [ ]:
# preds = LGBM.predict(test_x)

In [ ]:
# preds

In [ ]:
# stack_final

In [ ]:
# sub['공급량']